In [3]:
import pandas as pd
import hsfs
from hops import hdfs
connection = hsfs.connection(engine="hive")
fs = connection.get_feature_store()


Connected. Call `.close()` to terminate connection gracefully.


In [9]:
orders_df = pd.read_csv(hdfs.project_path() + "Jupyter/fs-notebooks/ecommerce/order.csv", index_col=0, infer_datetime_format=True, 
                        parse_dates=['order_purchase_timestamp', 'order_approved_at', 
                                     'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])

orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  datetime64[ns]
 5   order_delivered_carrier_date   99441 non-null  datetime64[ns]
 6   order_delivered_customer_date  99441 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.8+ MB


In [10]:
 print(orders_df['order_status'].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [11]:
orders_fg = fs.create_feature_group(name="orders",
                                        version=1,
                                        primary_key=['order_id'],
                                        online_enabled=True,
                                        description="Orders with status")
orders_fg.save(orders_df)

Configuring ingestion job...
Uploading Pandas dataframe...
Launching ingestion job...
Ingestion Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/hopsworks/#!/project/119/jobs


In [20]:
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

orders_fg = fs.get_feature_group(name="orders", version=1)
orders_df = orders_fg.read()

ct = ColumnTransformer([("order_status", preprocessing.OneHotEncoder(), [2])], remainder = 'passthrough')

ct = ct.fit(orders_df)

2022-03-06 10:25:19,498 INFO: USE `demo_fs_jim00000_featurestore`
2022-03-06 10:25:20,285 INFO: SELECT `fg0`.`order_id`, `fg0`.`customer_id`, `fg0`.`order_status`, `fg0`.`order_purchase_timestamp`, `fg0`.`order_approved_at`, `fg0`.`order_delivered_carrier_date`, `fg0`.`order_delivered_customer_date`, `fg0`.`order_estimated_delivery_date`
FROM `demo_fs_jim00000_featurestore`.`orders_1` `fg0`


In [16]:
help(df)

Help on ColumnTransformer in module sklearn.compose._column_transformer object:

class ColumnTransformer(sklearn.base.TransformerMixin, sklearn.utils.metaestimators._BaseComposition)
 |  ColumnTransformer(transformers, remainder='drop', sparse_threshold=0.3, n_jobs=None, transformer_weights=None, verbose=False)
 |  
 |  Applies transformers to columns of an array or pandas DataFrame.
 |  
 |  This estimator allows different columns or column subsets of the input
 |  to be transformed separately and the features generated by each transformer
 |  will be concatenated to form a single feature space.
 |  This is useful for heterogeneous or columnar data, to combine several
 |  feature extraction mechanisms or transformations into a single transformer.
 |  
 |  Read more in the :ref:`User Guide <column_transformer>`.
 |  
 |  .. versionadded:: 0.20
 |  
 |  Parameters
 |  ----------
 |  transformers : list of tuples
 |      List of (name, transformer, column(s)) tuples specifying the
 |    